In [1]:
%pip install peft wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import numpy as np
import os
from kaggle_secrets import UserSecretsClient
import wandb
from huggingface_hub import login

2025-05-19 08:57:01.315347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747645021.541477      71 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747645021.609890      71 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
user_secrets = UserSecretsClient()
os.environ['hf_token'] = user_secrets.get_secret("HF_TOKEN")
os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_TOKEN")
assert(os.getenv('hf_token'))
assert(os.getenv('WANDB_API_KEY'))

In [15]:
from datetime import datetime
import pytz

now_utc = datetime.now(pytz.utc)
now_colombo = now_utc.astimezone(pytz.timezone('Asia/Colombo'))
time_str = now_colombo.strftime('%Y-%b-%d_%H-%M-%S')
run_name = f'fft-{time_str}'
print(run_name)

fft-2025-May-19_14-29-25


In [5]:
wandb.login(key=os.getenv('WANDB_API_KEY'))
wandb.init(project="choreo-doc-ast-fft", name=run_name)

login(token=os.getenv('hf_token'))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rtweera (rtw-rtweera) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
# Load tokenizer and model
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Use mixed precision for efficiency
    device_map="auto"            # Automatically choose best device setup (NOTE: remove if causes problems)
)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [7]:
# # First set the default device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")


In [8]:
# Load tokenizer and model
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [9]:
# Load dataset
dataset = load_dataset("json", data_files="/kaggle/input/choreo-dataset/choreo_dataset.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
# Split the dataset into training and validation sets
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [11]:
# Preprocess and tokenize dataset
def preprocess_function(examples):
    # We are providing same instruction for every message, so tokenize only once (for optimization)
    system_instruction = examples['instruction'][0] # select 1st instruction

    # format each example
    formatted_texts = []
    for inp, out in zip(examples['input'], examples['output']):
        # Using consistent system message with varied user inputs and assistant outputs
        formatted = f"<|im_start|>system\n{system_instruction}<|im_end|>\n<|im_start|>user\n{inp}<|im_end|>\n<|im_start|>assistant\n{out}<|im_end|>"
        formatted_texts.append(formatted)
    
    tokenized = tokenizer(
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=1024,
        return_tensors="pt"
    )
    
    # Create labels (for causal LM, typically identical to input_ids)
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

In [12]:
# Apply preprocessing to datasets
tokenized_train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

Map:   0%|          | 0/607 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [13]:
# Determine whether to use fp16 or bf16 based on hardware support
bf16_available = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
print(f"BF16 support available: {bf16_available}")

BF16 support available: True


In [16]:

# Set up training arguments - simplified without DeepSpeed
training_args = TrainingArguments(
    output_dir="./qwen_choreo_full_ft",
    learning_rate=1e-5,  # Slightly higher learning rate since we're not using DeepSpeed
    per_device_train_batch_size=1,  # Very small batch size to avoid OOM errors
    gradient_accumulation_steps=16,  # Increased accumulation to compensate for smaller batch
    per_device_eval_batch_size=1,
    num_train_epochs=3,  # Reduced epochs for faster training
    weight_decay=0.01,
    logging_steps=1,
    logging_first_step=True,
    eval_strategy="steps",
    eval_steps=5,
    save_strategy="steps",
    save_steps=20,
    # Precision options
    bf16=bf16_available,  # Use BF16 if supported
    fp16=not bf16_available and torch.cuda.is_available(),  # Otherwise use FP16 if on CUDA
    logging_dir="./logs",
    load_best_model_at_end=True,
    save_total_limit=4,
    report_to="wandb",
    push_to_hub=True,
    hub_model_id=f"qwen-choreo-full-ft-{time_str}",
    run_name=run_name,
    # Memory optimization
    gradient_checkpointing=True,
    optim="adamw_torch",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    # Avoid device issues
    no_cuda=not torch.cuda.is_available(),  # Only disable CUDA if not available
    dataloader_num_workers=1,  # Reduced for stability
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
)


/tmp/ipykernel_71/4183412951.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
# Start training
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
5,9.991600,9.121798
10,2.211700,1.031554
15,0.780100,0.689568
20,0.688400,0.610856
25,0.416500,0.566626
30,0.502000,0.543311
35,0.493000,0.525434
40,0.511700,0.515352
45,0.511400,0.509992
50,0.466900,0.506179


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=111, training_loss=1.1991912792394828, metrics={'train_runtime': 4343.2913, 'train_samples_per_second': 0.419, 'train_steps_per_second': 0.026, 'total_flos': 3900894724227072.0, 'train_loss': 1.1991912792394828, 'epoch': 2.922570016474465})

In [18]:
# Save the model locally
print("Saving model...")
model.save_pretrained("./qwen_choreo_full_ft_final")
tokenizer.save_pretrained("./qwen_choreo_full_ft_final")

print("Pushing model to Hub...")
model.push_to_hub(f"qwen-choreo-full-ft-{time_str}-final-k3")
tokenizer.push_to_hub(f"qwen-choreo-full-ft-{time_str}-final-k3")


Saving model...
Pushing model to Hub...


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rtweera/qwen-choreo-full-ft-2025-May-19_14-29-25-final-k3/commit/a679f60bd15c0001ac84adae63cea6720bfc5201', commit_message='Upload tokenizer', commit_description='', oid='a679f60bd15c0001ac84adae63cea6720bfc5201', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rtweera/qwen-choreo-full-ft-2025-May-19_14-29-25-final-k3', endpoint='https://huggingface.co', repo_type='model', repo_id='rtweera/qwen-choreo-full-ft-2025-May-19_14-29-25-final-k3'), pr_revision=None, pr_num=None)

In [20]:
# Optionally, test the model with a sample
test_input = "How do I configure my applications to scale automatically in choreo?"
formatted_test = f"<|im_start|>user\n{test_input}<|im_end|>\n<|im_start|>assistant\n"
inputs = tokenizer(formatted_test, return_tensors="pt").to(model.device)

# Generate response
outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=1000,
    temperature=0.5,
    top_p=0.9,
    do_sample=True
)

# Decode and print response
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|im_start|>user
How do I configure my applications to scale automatically in choreo?<|im_end|>
<|im_start|>assistant
To configure your applications to scale automatically in Choreo, you can follow these steps:\n\n### 1. **Create a Job Definition**\n   - In the Choreo Console, go to the left navigation menu and click on **Jobs**.\n   - Click on the job for which you want to enable automatic scaling.\n   - On the right side of the page, click on **+ Create Job**.\n\n### 2. **Select Automatic Scaling**\n   - Choose **Automatic Scaling** from the dropdown menu that appears after selecting the job.\n\n### 3. **Configure Automatic Scaling Settings**\n   - You will be prompted to enter details about how you want the application to scale.\n     - **Scaling Mode**: Choose between `Auto`, `Manual`, or `Scheduled` based on your requirements.\n       - **Scale Factor**: Enter the desired number of instances.\n     - **Max Instances**: Enter the maximum number of instances allowed during the speci

In [21]:
# Optionally, test the model with a sample
test_input = "What is choreo?"
formatted_test = f"<|im_start|>user\n{test_input}<|im_end|>\n<|im_start|>assistant\n"
inputs = tokenizer(formatted_test, return_tensors="pt").to(model.device)

# Generate response
outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=1000,
    temperature=0.5,
    top_p=0.9,
    do_sample=True
)

# Decode and print response
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|im_start|>user
What is choreo?<|im_end|>
<|im_start|>assistant
Choreo is a cloud-native, containerized application framework that provides a scalable and flexible environment for building and deploying web applications. It was developed by Anthos, an open-source initiative that aims to create a single platform for all cloud-native applications. Choreo allows developers to build web services in a simple, declarative manner, enabling them to quickly deploy and manage their applications across different environments.<|im_end|>


In [23]:
# Optionally, test the model with a sample
test_input = "Is choreo a product by wso2"
formatted_test = f"<|im_start|>user\n{test_input}<|im_end|>\n<|im_start|>assistant\n"
inputs = tokenizer(formatted_test, return_tensors="pt").to(model.device)

# Generate response
outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=1000,
    temperature=0.5,
    top_p=0.9,
    do_sample=True
)

# Decode and print response
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|im_start|>user
Is choreo a product by wso2<|im_end|>
<|im_start|>assistant
Yes, Choreo is a managed service offered by WSO2. It provides various features and services to help organizations develop applications using APIs, which are often referred to as API-based applications or ABAs (Application-Building APIs). Here’s how it works in relation to WSO2:

1. **API Management**: Choreo allows you to manage your application's APIs effectively. You can define the endpoints for your API, configure security policies, and integrate with other WSO2 components like Identity and Access Management (IAM) and Service Bus.

2. **DevOps Integration**: With Choreo, you can easily deploy and manage your applications across multiple environments such as development, staging, and production. This helps in maintaining consistency and reducing deployment time.

3. **API Discovery and Management**: Choreo enables developers to discover and manage their APIs programmatically through an API discovery endpoint

In [24]:
# Optionally, test the model with a sample
test_input = "What is the concept of planes in chores by wso2"
formatted_test = f"<|im_start|>user\n{test_input}<|im_end|>\n<|im_start|>assistant\n"
inputs = tokenizer(formatted_test, return_tensors="pt").to(model.device)

# Generate response
outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=1000,
    temperature=0.5,
    top_p=0.9,
    do_sample=True
)

# Decode and print response
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|im_start|>user
What is the concept of planes in chores by wso2<|im_end|>
<|im_start|>assistant
In WSO2 Choreo, a plane is a logical component that defines how tasks are executed. It allows you to organize and manage different types of workloads within your workflow. Here's what you can do with planes:\n\n1. **Define Workload Types**: You can create various types of workloads such as "Manual," "Scheduled," or "External" based on their purpose.\n   - For example, you might have a "Manual" workload for manual data entry, a "Scheduled" workload for scheduled data processing, and an "External" workload for external data retrieval.\n\n2. **Assign Tasks to Workloads**: You can assign specific tasks to these workloads. This means that when you execute a task, it will be assigned to the appropriate workload type.\n   - For instance, if you want to perform a manual data entry task using the "Manual" workload, you would assign this task to the "Manual" workload.\n\n3. **Execute Tasks Based on W